# Imports and Libraries

In [1]:
from square.client import Client
import os
import pandas as pd

from google_sheets import *
from boto import get_square_secret
import datetime as dt
# import pandas as pd

# import os
import pickle
import uuid

In [2]:
client = Client(
    access_token=get_square_secret()['SQUARE_ACCESS_TOKEN'],
    environment='production')

TypeError: string indices must be integers, not 'str'

# Get OTPI Sheet

In [1]:
SAMPLE_SPREADSHEET_ID_input = '1lalOwWx_GwUZgjG3WgyxZvirtT9GYDJ6t3dAuNimTLM'
SAMPLE_RANGE_NAME = 'OTPI Scores!A:V'

opti_scores = get_spread_sheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME)



NameError: name 'get_spread_sheet' is not defined

# Pick Game and Team to generate invoices for

In [31]:
print(opti_scores['Game Date'].drop_duplicates())

0       5 Aug
63     12 Aug
132    19 Aug
212    26 Aug
279    02 Sep
350    09 Sep
432    16 Sep
515    23 Sep
597    30 Sep
678    07 Oct
774    14 Oct
856    21 Oct
913    28 Oct
Name: Game Date, dtype: object


In [32]:
game='28 Oct'
teams = ['1','2', '3','4', '5', '6']



invoices_to_raise = opti_scores[opti_scores['Game Date']==game]
invoices_to_raise = invoices_to_raise[invoices_to_raise['OT Team'].isin(teams)]
invoices_to_raise=invoices_to_raise[pd.to_numeric(invoices_to_raise['Amount Owed'])>0]
invoices_to_raise

,Game Date,Competition,Player Name,Position,OT Team,Opposition Name,Result,Goals,Assists,Goals Conceded,...,Red Cards,Clean Sheet?,Penalties Missed,Penalties Saved,Own Goals,MOTM?,Kit,Amount Owed,Match day rating,Comments
913,28 Oct,League,John Harrison,GK,1,Old Ignatians,WIN,,,1,...,,FALSE,,,,,,10,8,Cruitial contrubtion in the early exchanges of...
914,28 Oct,League,Aaron Snelling,DEF,1,Old Ignatians,WIN,,,1,...,,FALSE,,,,,,10,7.5,Overcovered far too much first half creating t...
915,28 Oct,League,Matt Crowe,DEF,1,Old Ignatians,WIN,2,,1,...,,FALSE,,,,1,,10,9.5,Monster of moster display. Ground defending s...
916,28 Oct,League,Theo Horton,DEF,1,Old Ignatians,WIN,,,1,...,,FALSE,,,,,,10,6.5,Rare poor showing. Gave the ball away a lot. ...
917,28 Oct,League,Jamie Wood,DEF,1,Old Ignatians,WIN,,,1,...,,FALSE,,,,,,10,7.5,So so game. Not at his best but didn't let us...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,28 Oct,League,Afnan Khan,DEF,6,Glyn Old Boys 2s,Loss,,,5,...,,FALSE,,,,,,5,5,late again. trying to turn and take players on...
989,28 Oct,League,Richard Scott,ATT,6,Glyn Old Boys 2s,Loss,,,5,...,,FALSE,,,,,,10,6.5,decent touches in first half up top then playe...
990,28 Oct,League,Matt Hope,ATT,6,Glyn Old Boys 2s,Loss,,,5,...,,FALSE,,,,,,10,6,too many touches. runs himself into trouble -s...
991,28 Oct,League,Jack Hind,ATT,6,Glyn Old Boys 2s,Loss,1,,5,...,,FALSE,,,,,,10,7,scored from a header from jeromes corner.had a...


In [33]:
pd.to_numeric(invoices_to_raise['Amount Owed']).sum()

665.5

# Get Current Debt

In [34]:
SAMPLE_SPREADSHEET_ID_input = '1K_uXqI1eiguBMr0uH3Tu7xDzePAIi3uAsDrAWukyBq8'
SAMPLE_RANGE_NAME = 'Overall Summary!B2:D1000'

matchday_debt = get_spread_sheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME)
matchday_debt = matchday_debt[matchday_debt['Name']!='']

matchday_debt

,Name,22-23 Debt,Match Day Debt
0,Aamish Ibrahi,£0.00,£0.00
1,Aaron Snelling,£0.00,£15.00
2,Abdul Adaawe,£0.00,£0.00
3,Abdul Hassan,£0.00,£0.00
4,Abz Marfa,£0.00,£5.00
...,...,...,...
310,Will French,£0.00,£5.00
311,Will Salt,£0.00,£0.00
312,Will Sollory,£0.00,£5.00
313,Yemi Oviosu,£0.00,£4.00


# Merge to Opti

In [35]:
invoice_mg_1 = pd.merge(left=invoices_to_raise, right = matchday_debt, how = 'left', right_on='Name', left_on = 'Player Name')
invoice_mg_1['Match Day Debt'] = pd.to_numeric(invoice_mg_1['Match Day Debt'].str.replace('£', ''))
invoice_mg = invoice_mg_1[invoice_mg_1['Match Day Debt']>0]
#
invoice_mg=invoice_mg[['Player Name', 'OT Team', 'Game Date', 'Opposition Name', 'Amount Owed']]

print('invoices to be made')
display(invoice_mg)
print(invoice_mg.shape)

print('up to date')
display(invoice_mg_1[invoice_mg_1['Match Day Debt']<=0][['Player Name', 'OT Team', 'Game Date', 'Opposition Name', 'Amount Owed']])
print(invoice_mg_1[invoice_mg_1['Match Day Debt']<=0][['Player Name', 'OT Team', 'Game Date', 'Opposition Name', 'Amount Owed']].shape)

invoices to be made


,Player Name,OT Team,Game Date,Opposition Name,Amount Owed
1,Aaron Snelling,1,28 Oct,Old Ignatians,10
4,Jamie Wood,1,28 Oct,Old Ignatians,10
5,Ollie Sewell,1,28 Oct,Old Ignatians,2.5
6,Yemi Oviosu,1,28 Oct,Old Ignatians,10
8,Keaton Cohen,1,28 Oct,Old Ignatians,10
9,Jarvis Cheek,1,28 Oct,Old Ignatians,10
10,Adam Platten,1,28 Oct,Old Ignatians,5
13,Luke Dyges,1,28 Oct,Old Ignatians,3
14,Andrew Barclay,2,28 Oct,Wandsworth Borough 2s,10
15,Max Watson,2,28 Oct,Wandsworth Borough 2s,10


(43, 5)
up to date


,Player Name,OT Team,Game Date,Opposition Name,Amount Owed
0,John Harrison,1,28 Oct,Old Ignatians,10
2,Matt Crowe,1,28 Oct,Old Ignatians,10
3,Theo Horton,1,28 Oct,Old Ignatians,10
7,Josh Sherriff-Jolly,1,28 Oct,Old Ignatians,10
11,Stanley Timms,1,28 Oct,Old Ignatians,5
12,James Rodriguez,1,28 Oct,Old Ignatians,5
16,Jonny Simmons,2,28 Oct,Wandsworth Borough 2s,10
18,Finn Coolen,2,28 Oct,Wandsworth Borough 2s,5
23,Paul Ball,2,28 Oct,Wandsworth Borough 2s,10
25,Deji Ogunyemi,2,28 Oct,Wandsworth Borough 2s,10


(34, 5)


In [36]:
invoice_mg_1

,Game Date,Competition,Player Name,Position,OT Team,Opposition Name,Result,Goals,Assists,Goals Conceded,...,Penalties Saved,Own Goals,MOTM?,Kit,Amount Owed,Match day rating,Comments,Name,22-23 Debt,Match Day Debt
0,28 Oct,League,John Harrison,GK,1,Old Ignatians,WIN,,,1,...,,,,,10,8,Cruitial contrubtion in the early exchanges of...,John Harrison,£0.00,-10.0
1,28 Oct,League,Aaron Snelling,DEF,1,Old Ignatians,WIN,,,1,...,,,,,10,7.5,Overcovered far too much first half creating t...,Aaron Snelling,£0.00,15.0
2,28 Oct,League,Matt Crowe,DEF,1,Old Ignatians,WIN,2,,1,...,,,1,,10,9.5,Monster of moster display. Ground defending s...,Matt Crowe,£0.00,-12.0
3,28 Oct,League,Theo Horton,DEF,1,Old Ignatians,WIN,,,1,...,,,,,10,6.5,Rare poor showing. Gave the ball away a lot. ...,Theo Horton,£0.00,0.0
4,28 Oct,League,Jamie Wood,DEF,1,Old Ignatians,WIN,,,1,...,,,,,10,7.5,So so game. Not at his best but didn't let us...,Jamie Wood,£0.00,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,28 Oct,League,Afnan Khan,DEF,6,Glyn Old Boys 2s,Loss,,,5,...,,,,,5,5,late again. trying to turn and take players on...,Afnan Khan,£0.00,5.0
73,28 Oct,League,Richard Scott,ATT,6,Glyn Old Boys 2s,Loss,,,5,...,,,,,10,6.5,decent touches in first half up top then playe...,Richard Scott,£0.00,0.0
74,28 Oct,League,Matt Hope,ATT,6,Glyn Old Boys 2s,Loss,,,5,...,,,,,10,6,too many touches. runs himself into trouble -s...,Matt Hope,£0.00,0.0
75,28 Oct,League,Jack Hind,ATT,6,Glyn Old Boys 2s,Loss,1,,5,...,,,,,10,7,scored from a header from jeromes corner.had a...,Jack Hind,£0.00,0.0


# Get Customer IDs 

In [37]:
customers_api = client.customers

In [39]:

result = customers_api.list_customers(
    
)
customers = {str(cust.get('given_name')) + ' ' + str(cust.get('family_name')):cust.get('id') for cust in result.body['customers']}


i=1
while result.cursor:
    print(i)
    i+=1
    result = customers_api.list_customers(
        cursor = result.cursor
    )
    customers2 = {str(cust.get('given_name')) + ' ' + str(cust.get('family_name')):cust.get('id') for cust in result.body['customers']}

    customers.update(customers2)




1
2
3
4
5
6
7
8


In [40]:
customer_df=pd.DataFrame(data=[{'name':cust, 'id':customers[cust]} for cust in customers])

In [41]:
customer_df.groupby(['name']).count().reset_index().sort_values('id')

,name,id
0,Aaron Snelling,1
507,Mr Smalley,1
508,Mr Smithson,1
509,Mr Washington,1
510,Mr Wastie,1
...,...,...
260,Isaac Aidoo,1
261,Ismail Afenyo,1
262,Ismail Wall,1
192,Ellis Forrester,1


In [42]:
customer_df

,name,id
0,Aaron Snelling,0VBREWQGYS5G3D5764XW4TZH98
1,Abdul Hassan,XH3400YABCPV8RJKCWEJDA8X0M
2,Abimbola Ogunyemi,1R5JPH26RN1FSF79HNF9KM1NMC
3,Abz Marfa,2PFFC075PXKEPWSR1Y5AHXH25G
4,Adam Bouhali,3EBPJ89VGFZTN61SD7KR66K1Y8
...,...,...
765,Zachary Williams,TF1ZAX80D09ST3J7PK4PGY0RN8
766,Zakaria Abdullahi,FJEYM95DSZTAX7DYTXXT9FYC3C
767,Zaki Yusuf,66AT39ABC29XNX1FX702Z6BH8C
768,Zakriya Gul,V295YNGJSD4235PXWGMEVKK088


In [43]:
id_mg = pd.merge(left = invoice_mg, right = customer_df, how= 'left', left_on = 'Player Name', right_on = 'name')

id_mg['Amount Owed']=pd.to_numeric(id_mg['Amount Owed'])



id_mg

,Player Name,OT Team,Game Date,Opposition Name,Amount Owed,name,id
0,Aaron Snelling,1,28 Oct,Old Ignatians,10.0,Aaron Snelling,0VBREWQGYS5G3D5764XW4TZH98
1,Jamie Wood,1,28 Oct,Old Ignatians,10.0,Jamie Wood,XG84PQRK9D4BH7FD3WNM6R818G
2,Ollie Sewell,1,28 Oct,Old Ignatians,2.5,Ollie Sewell,D4DTBQAP7HX04K14EZ47CWWR70
3,Yemi Oviosu,1,28 Oct,Old Ignatians,10.0,Yemi Oviosu,KRVP85KPCH7Y77HPR47JN3WFPC
4,Keaton Cohen,1,28 Oct,Old Ignatians,10.0,Keaton Cohen,YZ989HEH5S0YK90RW4J3T6DTC8
5,Jarvis Cheek,1,28 Oct,Old Ignatians,10.0,Jarvis Cheek,BYJH2ZRDSH122PJHGD2CWW0ZRR
6,Adam Platten,1,28 Oct,Old Ignatians,5.0,Adam Platten,TC0C4DZ3Y217B8Y8A1EZ0PV4F0
7,Luke Dyges,1,28 Oct,Old Ignatians,3.0,Luke Dyges,M2WN0A8Z056ECKKPCZ3M1GY88R
8,Andrew Barclay,2,28 Oct,Wandsworth Borough 2s,10.0,Andrew Barclay,AP38QWE4417ZAYSY1RWV4N16K8
9,Max Watson,2,28 Oct,Wandsworth Borough 2s,10.0,Max Watson,4KEVYTZKFD01B4NE8KA0N1X8Q4


In [44]:
id_mg['Amount Owed'].sum()

372.5

# Create Orders

In [45]:
orders_api = client.orders

new_orders = []

for i, row in id_mg.iterrows():
    print(i)
    amount = row['Amount Owed'] 
    body = {
        'order': {
            'location_id': 'SBBJSGR24YHYB',
            'reference_id': f'{uuid.uuid1()}',
            'line_items': [
                
                {
                    'quantity': '1',
                    'catalog_object_id': 'ORCOS6EPTCLMYSWEMPX2PBUK',
                    'base_price_money': {
                        'amount': amount*100,
                        'currency': 'GBP'
                    }
                }
            ]
        },
        'idempotency_key': f'{uuid.uuid1()}'
    }
    # print(body)
    
    result = orders_api.create_order(body)
    # print(result)
    obj_id=result.body['order']['id']

    new_orders.append({'Player Name':row['Player Name'], 'obj_id':obj_id})
    # # print(result)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [46]:
order_mg= pd.merge(left = id_mg, right=pd.DataFrame(new_orders), how= 'left', on = 'Player Name')

order_mg

,Player Name,OT Team,Game Date,Opposition Name,Amount Owed,name,id,obj_id
0,Aaron Snelling,1,28 Oct,Old Ignatians,10.0,Aaron Snelling,0VBREWQGYS5G3D5764XW4TZH98,JvVA1SRadPZEoGIrDToyf2OE3RPZY
1,Jamie Wood,1,28 Oct,Old Ignatians,10.0,Jamie Wood,XG84PQRK9D4BH7FD3WNM6R818G,XIu2eH2tBYSuVwhHPq2QcDhUuODZY
2,Ollie Sewell,1,28 Oct,Old Ignatians,2.5,Ollie Sewell,D4DTBQAP7HX04K14EZ47CWWR70,ppoJaYs6E0JXT6XMLeNABbyC96KZY
3,Yemi Oviosu,1,28 Oct,Old Ignatians,10.0,Yemi Oviosu,KRVP85KPCH7Y77HPR47JN3WFPC,59GdCIlDk8eTXPckQnNrCtotZ4HZY
4,Keaton Cohen,1,28 Oct,Old Ignatians,10.0,Keaton Cohen,YZ989HEH5S0YK90RW4J3T6DTC8,FF5205C42rlMjntyAxArhY8RfTRZY
5,Jarvis Cheek,1,28 Oct,Old Ignatians,10.0,Jarvis Cheek,BYJH2ZRDSH122PJHGD2CWW0ZRR,Dg9dKv05vfa282cqVnKyRAS1ydUZY
6,Adam Platten,1,28 Oct,Old Ignatians,5.0,Adam Platten,TC0C4DZ3Y217B8Y8A1EZ0PV4F0,tHyp3U1PD7D6oPNBhpzaReNKmLAZY
7,Luke Dyges,1,28 Oct,Old Ignatians,3.0,Luke Dyges,M2WN0A8Z056ECKKPCZ3M1GY88R,9b8REpDGxPP3YVpqdb69ISCkJrMZY
8,Andrew Barclay,2,28 Oct,Wandsworth Borough 2s,10.0,Andrew Barclay,AP38QWE4417ZAYSY1RWV4N16K8,pHnrhD3n5LnLVRM5jMXRQo8KZFNZY
9,Max Watson,2,28 Oct,Wandsworth Borough 2s,10.0,Max Watson,4KEVYTZKFD01B4NE8KA0N1X8Q4,hJ65OQnYjMWKXY1B9ysTzyD3emaZY


# Create Invoices

In [47]:
invoices_api = client.invoices

In [48]:
due_at =  str(dt.datetime.now().date())
hour_due=dt.datetime.now().hour+1
hour_due

13

In [49]:
invoices = []

for i, row in order_mg.iterrows():
    order_id = row['obj_id']
    cust_id = row['id']
    invoice_number = str(dt.datetime.now().date()) +f"_{row['OT Team']}"+f"_{row['obj_id']}_1"

    

    title = 'Match Subs ' + row['Game Date'] + ' vs ' + row['Opposition Name']

        
    body = {
    'invoice': {
        'location_id': 'SBBJSGR24YHYB',
        'order_id': f'{order_id}',
        'primary_recipient': {
            'customer_id': f'{cust_id}'
        },
        'payment_requests': [
            {
                'request_type': 'BALANCE',
                'due_date': due_at,
                'tipping_enabled': True,
                'automatic_payment_source': 'NONE',
                'reminders': [
                    {
                        'relative_scheduled_days': -1,
                        'message': 'Your Invoice is due'
                    }
                ]
            }
        ],
        'delivery_method': 'EMAIL',
        'invoice_number': invoice_number,
        'title': title,
        'description': title,
        'scheduled_at': due_at+f'T{hour_due}:00:00Z',
        'accepted_payment_methods': {
            'card': True,
            'square_gift_card': False,
            'bank_account': False,
            'buy_now_pay_later': False,
            'cash_app_pay': False
        },
        'sale_or_service_date': str(dt.datetime.now().date()),
        'store_payment_method_enabled': False
    },
    'idempotency_key': f'{uuid.uuid1()}'
    }

    result = invoices_api.create_invoice(body)
    print(row['Player Name'])
    print(result)
    invidid=result.body['invoice']['id']
    invoices.append(invidid)

Aaron Snelling
<ApiResponse {"invoice":{"id":"inv:0-ChDfASK6S50RCMI7mZgaaZd7EIMP","version":0,"location_id":"SBBJSGR24YHYB","order_id":"JvVA1SRadPZEoGIrDToyf2OE3RPZY","payment_requests":[{"uid":"e4f730e6-4e5d-4bee-87d1-a6f2791cdc98","request_type":"BALANCE","due_date":"2023-10-31","tipping_enabled":true,"computed_amount_money":{"amount":1000,"currency":"GBP"},"total_completed_amount_money":{"amount":0,"currency":"GBP"},"reminders":[{"uid":"aed10ed2-d05e-4092-bab4-3833b706cfce","relative_scheduled_days":-1,"message":"Your Invoice is due","status":"PENDING"}],"automatic_payment_source":"NONE"}],"primary_recipient":{"customer_id":"0VBREWQGYS5G3D5764XW4TZH98","given_name":"Aaron","family_name":"Snelling","email_address":"aaron.s.c.s@hotmail.co.uk","phone_number":"+447984654583"},"invoice_number":"2023-10-31_1_JvVA1SRadPZEoGIrDToyf2OE3RPZY_1","title":"Match Subs 28 Oct vs Old Ignatians","description":"Match Subs 28 Oct vs Old Ignatians","scheduled_at":"2023-10-31T13:00:00Z","status":"DRAFT"

In [50]:
for invidid in invoices:

    body = {
        'version': 0,
        'idempotency_key': f'{invidid}'
    }

    result = invoices_api.publish_invoice(
        invidid,
        body
    )
    print(result)

<ApiResponse {"invoice":{"id":"inv:0-ChDfASK6S50RCMI7mZgaaZd7EIMP","version":1,"location_id":"SBBJSGR24YHYB","order_id":"JvVA1SRadPZEoGIrDToyf2OE3RPZY","payment_requests":[{"uid":"e4f730e6-4e5d-4bee-87d1-a6f2791cdc98","request_type":"BALANCE","due_date":"2023-10-31","tipping_enabled":true,"computed_amount_money":{"amount":1000,"currency":"GBP"},"total_completed_amount_money":{"amount":0,"currency":"GBP"},"reminders":[{"uid":"aed10ed2-d05e-4092-bab4-3833b706cfce","relative_scheduled_days":-1,"message":"Your Invoice is due","status":"NOT_APPLICABLE"}],"automatic_payment_source":"NONE"}],"primary_recipient":{"customer_id":"0VBREWQGYS5G3D5764XW4TZH98","given_name":"Aaron","family_name":"Snelling","email_address":"aaron.s.c.s@hotmail.co.uk","phone_number":"+447984654583"},"invoice_number":"2023-10-31_1_JvVA1SRadPZEoGIrDToyf2OE3RPZY_1","title":"Match Subs 28 Oct vs Old Ignatians","description":"Match Subs 28 Oct vs Old Ignatians","scheduled_at":"2023-10-31T13:00:00Z","status":"SCHEDULED","ti